In [12]:
%matplotlib inline

import keras
import os
import json
from datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np


from PIL import Image
from tensorflow.python.ops import array_ops
from tqdm.notebook import tqdm
from random import shuffle
from glob import glob


from keras import backend as K
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Cropping2D, Lambda, GlobalAveragePooling2D,BatchNormalization,AveragePooling2D
from keras.models import Sequential,Model
from keras.optimizers import SGD, rmsprop, Adam
from keras.metrics import categorical_accuracy, categorical_crossentropy, top_k_categorical_accuracy
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model, model_from_json

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

# Import main model architecture

from keras.applications.densenet import DenseNet121, preprocess_input 

pd.options.display.max_columns = 200

ModuleNotFoundError: No module named 'tqdm_notebook'

In [2]:
train_paths = glob(r"E:\crew2_train\training\*")
train_idx = [path.split("-")[-1].split(".")[0] for path in train_paths]

train_len = len(train_paths)

MODEL_NAME = 'first_batch_model'

target_names = ['W','S','A','D','WA','WD','SA','SD','NK']

In [3]:
WIDTH = 480
HEIGHT = 270
LR = 1e-3
EPOCHS = 30
TEST_SIZE = .2
BS = 8

In [4]:
base_model = DenseNet121(include_top=False, weights='imagenet',
                         input_shape=(HEIGHT, WIDTH, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(.2)(x)

predictions = Dense(9, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

## Init with imagenet & finetune:

for layer in model.layers:  
    layer.trainable = True

model.compile(optimizer=Adam(lr=1e-4, decay=1e-9), loss="categorical_crossentropy", metrics=["accuracy", top_3_accuracy, categorical_accuracy])

In [5]:
# For tensorboard: $ tensorboard --logdir=logs/scalars/

In [6]:
shuffle(train_idx)

for l in range(8):
    for count, k in enumerate(train_idx):

        # logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
        # tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
        # callbacks = [EarlyStopping(patience = 4, verbose = 1),
        #            ReduceLROnPlateau(factor = 0.1, patience = 4, min_lr = 0.00001, verbose = 1)]
        
        # callbacks = [EarlyStopping(patience = 4, verbose = 1),
        #            ReduceLROnPlateau(factor = 0.1, patience = 4, min_lr = 0.00001, verbose = 1), tensorboard_callback]
        
        data_npy_path = 'E:/crew2_train/training/training_data-{}.npy'.format(train_idx[count])
        df = np.load(data_npy_path)

        print('\nLoaded file: training_data-{}.npy'.format(k) + ", Length: " +str(len(df)) + "\n")

        df_feature = np.array([i[0] for i in df])
        df_label = [i[1] for i in df]    

        del df

        df_feature = preprocess_input(df_feature) 
        x_train, x_test, y_train, y_test = train_test_split(df_feature, df_label, test_size=TEST_SIZE)

        del df_feature, df_label

        # Construct the training image generator for data augmentation

        aug = ImageDataGenerator()

        aug_val = ImageDataGenerator()

        # Train the network
        H = model.fit(x_train, np.array(y_train),
                    batch_size = BS,
                    epochs = 1,
                    validation_data=(x_test, np.array(y_test)))
        #  H = model.fit_generator(aug.flow(x_train, y_train, batch_size = BS),
        #                          validation_data = aug_val.flow(x_test, y_test, batch_size = BS) , 
        #                          validation_steps = ((len(y_train) + len(y_test))//TEST_SIZE)// BS,
        #                          #callbacks = callbacks, 
        #                          steps_per_epoch = (len(y_train) + len(y_test)) // BS,
        #                          workers = 8,
        #                          epochs = 1)

        print("Batch {} is finished.\n".format(count))

        # plt.figure(figsize=(8, 8))
        # plt.title("Learning curve")
        # plt.plot(H.history["loss"], label="loss")
        # plt.plot(H.history["val_loss"], label="val_loss")
        # plt.plot( np.argmin(H.history["val_loss"]), np.min(H.history["val_loss"]), marker="x", color="r", label="best model")
        # plt.xlabel("Epochs")
        # plt.ylabel("CE Loss")
        # plt.legend()
        # plt.show()

        #  out = model.predict(x_test)
        #  y_pred1 = np.argmax(out, axis=-1)

        #  print('\nConfusion Matrix')
        #  print(confusion_matrix(np.argmax(y_test, axis=-1), y_pred1))
        #  print('\nAccuracy: ',accuracy_score(np.argmax(y_test, axis=-1), y_pred1))
        
    print("\nEpoch {} - Saving Model.\n".format(l))
    
    ## Reload weights & model structure:
    
    model.save_weights(MODEL_NAME + '_Epoch_{}.h5'.format(l))
    model_json = model.to_json()
    with open(MODEL_NAME + '_Epoch_{}_'.format(l) + "json.json", "w") as json_file:
        json.dump(model_json, json_file)
    


Loaded file: training_data-86.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 57s 142ms/step - loss: 1.3379 - acc: 0.5325 - top_3_accuracy: 0.8475 - categorical_accuracy: 0.5325 - val_loss: 0.9322 - val_acc: 0.6100 - val_top_3_accuracy: 0.9500 - val_categorical_accuracy: 0.6100
Batch 0 is finished.


Loaded file: training_data-131.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 25s 61ms/step - loss: 1.3286 - acc: 0.5300 - top_3_accuracy: 0.8100 - categorical_accuracy: 0.5300 - val_loss: 1.0365 - val_acc: 0.6300 - val_top_3_accuracy: 0.8600 - val_categorical_accuracy: 0.6300
Batch 1 is finished.


Loaded file: training_data-85.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 24s 61ms/step - loss: 1.1604 - acc: 0.5575 - top_3_accuracy: 0.8900 - categorical_accuracy: 0.5575 - val_loss:

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 24s 61ms/step - loss: 1.2246 - acc: 0.5550 - top_3_accuracy: 0.8725 - categorical_accuracy: 0.5550 - val_loss: 0.9394 - val_acc: 0.6300 - val_top_3_accuracy: 0.9000 - val_categorical_accuracy: 0.6300
Batch 22 is finished.


Loaded file: training_data-158.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 24s 61ms/step - loss: 0.7925 - acc: 0.7050 - top_3_accuracy: 0.9625 - categorical_accuracy: 0.7050 - val_loss: 0.6400 - val_acc: 0.7900 - val_top_3_accuracy: 0.9800 - val_categorical_accuracy: 0.7900
Batch 23 is finished.


Loaded file: training_data-174.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 24s 61ms/step - loss: 0.5145 - acc: 0.8550 - top_3_accuracy: 0.9675 - categorical_accuracy: 0.8550 - val_loss: 0.6796 - val_acc: 0.7800 - val_top_3_accuracy:

400/400 [==============================] - 24s 61ms/step - loss: 1.0904 - acc: 0.5325 - top_3_accuracy: 0.9350 - categorical_accuracy: 0.5325 - val_loss: 1.1377 - val_acc: 0.4300 - val_top_3_accuracy: 0.9900 - val_categorical_accuracy: 0.4300
Batch 44 is finished.


Loaded file: training_data-173.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 24s 61ms/step - loss: 0.5428 - acc: 0.8525 - top_3_accuracy: 0.9975 - categorical_accuracy: 0.8525 - val_loss: 0.5699 - val_acc: 0.8300 - val_top_3_accuracy: 1.0000 - val_categorical_accuracy: 0.8300
Batch 45 is finished.


Loaded file: training_data-113.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 24s 61ms/step - loss: 0.8492 - acc: 0.6800 - top_3_accuracy: 0.9575 - categorical_accuracy: 0.6800 - val_loss: 0.7831 - val_acc: 0.7600 - val_top_3_accuracy: 0.9200 - val_categorical_accuracy: 0.7600
Batch 46 is f

400/400 [==============================] - 24s 61ms/step - loss: 1.0317 - acc: 0.5925 - top_3_accuracy: 0.9275 - categorical_accuracy: 0.5925 - val_loss: 1.0589 - val_acc: 0.6200 - val_top_3_accuracy: 0.9000 - val_categorical_accuracy: 0.6200
Batch 66 is finished.


Loaded file: training_data-90.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 24s 61ms/step - loss: 1.2787 - acc: 0.5525 - top_3_accuracy: 0.8750 - categorical_accuracy: 0.5525 - val_loss: 0.9027 - val_acc: 0.6700 - val_top_3_accuracy: 0.9700 - val_categorical_accuracy: 0.6700
Batch 67 is finished.


Loaded file: training_data-122.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 24s 60ms/step - loss: 0.9408 - acc: 0.6800 - top_3_accuracy: 0.9625 - categorical_accuracy: 0.6800 - val_loss: 0.7974 - val_acc: 0.7300 - val_top_3_accuracy: 0.9900 - val_categorical_accuracy: 0.7300
Batch 68 is fi

400/400 [==============================] - 24s 61ms/step - loss: 0.7790 - acc: 0.7100 - top_3_accuracy: 0.9775 - categorical_accuracy: 0.7100 - val_loss: 0.6991 - val_acc: 0.6800 - val_top_3_accuracy: 1.0000 - val_categorical_accuracy: 0.6800
Batch 88 is finished.


Loaded file: training_data-168.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 24s 61ms/step - loss: 0.5447 - acc: 0.7825 - top_3_accuracy: 0.9950 - categorical_accuracy: 0.7825 - val_loss: 0.5426 - val_acc: 0.7900 - val_top_3_accuracy: 0.9900 - val_categorical_accuracy: 0.7900
Batch 89 is finished.


Loaded file: training_data-37.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 24s 61ms/step - loss: 1.0869 - acc: 0.6400 - top_3_accuracy: 0.9275 - categorical_accuracy: 0.6400 - val_loss: 0.8676 - val_acc: 0.6900 - val_top_3_accuracy: 0.9500 - val_categorical_accuracy: 0.6900
Batch 90 is fi

...

400/400 [==============================] - 24s 61ms/step - loss: 0.6931 - acc: 0.7400 - top_3_accuracy: 0.9750 - categorical_accuracy: 0.7400 - val_loss: 0.8112 - val_acc: 0.6300 - val_top_3_accuracy: 0.9600 - val_categorical_accuracy: 0.6300
Batch 54 is finished.


Loaded file: training_data-161.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 24s 61ms/step - loss: 0.4099 - acc: 0.8400 - top_3_accuracy: 1.0000 - categorical_accuracy: 0.8400 - val_loss: 0.3317 - val_acc: 0.8600 - val_top_3_accuracy: 1.0000 - val_categorical_accuracy: 0.8600
Batch 55 is finished.


Loaded file: training_data-163.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 24s 60ms/step - loss: 0.4439 - acc: 0.8025 - top_3_accuracy: 1.0000 - categorical_accuracy: 0.8025 - val_loss: 0.6088 - val_acc: 0.7800 - val_top_3_accuracy: 1.0000 - val_categorical_accuracy: 0.7800
Batch 56 is f

400/400 [==============================] - 24s 61ms/step - loss: 0.3953 - acc: 0.8250 - top_3_accuracy: 1.0000 - categorical_accuracy: 0.8250 - val_loss: 0.4085 - val_acc: 0.7700 - val_top_3_accuracy: 1.0000 - val_categorical_accuracy: 0.7700
Batch 76 is finished.


Loaded file: training_data-63.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 24s 61ms/step - loss: 0.4097 - acc: 0.8225 - top_3_accuracy: 1.0000 - categorical_accuracy: 0.8225 - val_loss: 0.3454 - val_acc: 0.8400 - val_top_3_accuracy: 1.0000 - val_categorical_accuracy: 0.8400
Batch 77 is finished.


Loaded file: training_data-170.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 24s 61ms/step - loss: 0.3100 - acc: 0.8675 - top_3_accuracy: 0.9975 - categorical_accuracy: 0.8675 - val_loss: 0.3095 - val_acc: 0.8300 - val_top_3_accuracy: 1.0000 - val_categorical_accuracy: 0.8300
Batch 78 is fi

400/400 [==============================] - 25s 61ms/step - loss: 0.5779 - acc: 0.7800 - top_3_accuracy: 0.9875 - categorical_accuracy: 0.7800 - val_loss: 0.7410 - val_acc: 0.6600 - val_top_3_accuracy: 0.9700 - val_categorical_accuracy: 0.6600
Batch 98 is finished.


Loaded file: training_data-19.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 25s 61ms/step - loss: 0.6267 - acc: 0.6875 - top_3_accuracy: 1.0000 - categorical_accuracy: 0.6875 - val_loss: 0.5102 - val_acc: 0.7500 - val_top_3_accuracy: 1.0000 - val_categorical_accuracy: 0.7500
Batch 99 is finished.


Loaded file: training_data-166.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 25s 61ms/step - loss: 0.5493 - acc: 0.7625 - top_3_accuracy: 0.9975 - categorical_accuracy: 0.7625 - val_loss: 0.3871 - val_acc: 0.8700 - val_top_3_accuracy: 1.0000 - val_categorical_accuracy: 0.8700
Batch 100 is f

400/400 [==============================] - 25s 61ms/step - loss: 0.6917 - acc: 0.7050 - top_3_accuracy: 0.9825 - categorical_accuracy: 0.7050 - val_loss: 0.7941 - val_acc: 0.6900 - val_top_3_accuracy: 0.9800 - val_categorical_accuracy: 0.6900
Batch 120 is finished.


Loaded file: training_data-186.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 25s 61ms/step - loss: 0.2319 - acc: 0.9175 - top_3_accuracy: 0.9925 - categorical_accuracy: 0.9175 - val_loss: 0.0887 - val_acc: 0.9800 - val_top_3_accuracy: 1.0000 - val_categorical_accuracy: 0.9800
Batch 121 is finished.


Loaded file: training_data-5.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 25s 62ms/step - loss: 0.5791 - acc: 0.7250 - top_3_accuracy: 0.9975 - categorical_accuracy: 0.7250 - val_loss: 0.7298 - val_acc: 0.6700 - val_top_3_accuracy: 0.9900 - val_categorical_accuracy: 0.6700
Batch 122 is 

400/400 [==============================] - 25s 62ms/step - loss: 0.4474 - acc: 0.8350 - top_3_accuracy: 0.9925 - categorical_accuracy: 0.8350 - val_loss: 0.6751 - val_acc: 0.8100 - val_top_3_accuracy: 1.0000 - val_categorical_accuracy: 0.8100
Batch 142 is finished.


Loaded file: training_data-36.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 25s 62ms/step - loss: 0.5377 - acc: 0.7425 - top_3_accuracy: 0.9950 - categorical_accuracy: 0.7425 - val_loss: 0.6626 - val_acc: 0.7300 - val_top_3_accuracy: 0.9700 - val_categorical_accuracy: 0.7300
Batch 143 is finished.


Loaded file: training_data-202.npy, Length: 500

Train on 400 samples, validate on 100 samples
Epoch 1/1
400/400 [==============================] - 25s 62ms/step - loss: 0.3653 - acc: 0.8600 - top_3_accuracy: 1.0000 - categorical_accuracy: 0.8600 - val_loss: 0.2577 - val_acc: 0.8900 - val_top_3_accuracy: 1.0000 - val_categorical_accuracy: 0.8900
Batch 144 is

## Create postprocessing weights

In [9]:
'''
After noticing such a substantial improvment 2.6% after my own manual changes,
validating my expectations, i've decided to automate this process.
'''

validation_dir = r"E:\crew2_train\validation"

USE_WEIGHTS = True


DECAY = 0.9
#          w0     s1     a2    d3    wa4     wd5   sa6     sd7     nk8
WEIGHTS = [1.0,  1.0,   1.0,  1.0,   1.0,    1.0,  1.0,    1.0,    1.0]

#WEIGHTS =[0.030903154382632643, 1000.0, 0.020275559590445278, 0.013302794647291147, 0.0225283995449392, 0.025031555049932444, 1000.0, 1000.0, 0.016423203268260675]


mapping_dict = {0: "W",
                1: "S",
                2: "A",
                3: "D",
                4: "WA",
                5: "WD",
                6: "SA",
                7: "SD",
                8: "NK",}


close_dict = {
              0: {4: 0.3, 5: 0.3, 8: 0.05},  # Should be W, but said WA or WD NK nbd
              1: {6: 0.3, 7: 0.3, 8: 0.05},  # Should be S, but said SA OR SD, NK nbd
              2: {4: 0.3, 6: 0.3},           # Should be A, but SA or WA
              3: {5: 0.3, 7: 0.3},           # Shoudl be D, but SD or WD
              4: {2: 0.5},                   # Should be WA, but A
              5: {3: 0.5},                   # Should be WD, but D
              6: {1: 0.3, 2: 0.3},           # Should be SA, but S or A
              7: {1: 0.3, 3: 0.3},           # Should be SD, but S or D
              8: {0: 0.05, 1: 0.05, 2: 0.05, 3: 0.05, 4: 0.05, 5: 0.05, 6: 0.05, 7: 0.05, },  # should be NK... but whatever.
              }


# with open('first_batch_model_Epoch_7_json.json','r') as f:
#     model_json = json.load(f)

# model = model_from_json(model_json)
# model.load_weights('first_batch_model_Epoch_7.h5')

In [10]:
dist_dict = {0: 0,
             1: 0,
             2: 0,
             3: 0,
             4: 0,
             5: 0,
             6: 0,
             7: 0,
             8: 0}

total = 0
correct = 0
closeness = 0
# step 1

In [15]:
for f in tqdm(os.listdir(validation_dir)):
    if ".npy" in f:

        print(os.path.join(validation_dir, f))
        chunk = np.load(os.path.join(validation_dir, f))

        for data in tqdm(chunk):
            total += 1
            X = np.array(data[0])
            X = preprocess_input(X)
            y = data[1]
#                 print(X.shape, len(y))
            prediction = model.predict([X.reshape(-1, X.shape[0], X.shape[1], X.shape[2])])[0]
            if USE_WEIGHTS:
                prediction = np.array(prediction) * np.array(WEIGHTS)

            dist_dict[np.argmax(prediction)] += 1

            if np.argmax(prediction) == np.argmax(y):
                correct += 1
                closeness += 1
            else:
                if np.argmax(prediction) in close_dict[np.argmax(y)]:
                    closeness += close_dict[np.argmax(y)][np.argmax(prediction)]
print(30*"_")
print("Weights:", WEIGHTS)
print(f"Epoch7 weights' accuracy: {round(correct/total, 3)}. Accuracy considering 'closeness': {round(closeness/total, 3)}")
print(dist_dict)
largest_key = max(dist_dict, key=dist_dict.get)

with open('log.txt', "a") as f:
    f.write("Weights: "+str(WEIGHTS))
    f.write('\n')
    f.write(f"Epoch7 weights' accuracy: {round(correct/total, 3)}. Accuracy considering 'closeness': {round(closeness/total, 3)}\n")
    f.write("Distribution: "+str(dist_dict))
    f.write("\n")
    f.write("\n")
    f.write("\n")

WEIGHTS[largest_key] *= DECAY

E:\crew2_train\validation\training_data-104.npy



E:\crew2_train\validation\training_data-107.npy



E:\crew2_train\validation\training_data-115.npy



E:\crew2_train\validation\training_data-116.npy



E:\crew2_train\validation\training_data-12.npy


...


E:\crew2_train\validation\training_data-82.npy



E:\crew2_train\validation\training_data-93.npy



E:\crew2_train\validation\training_data-94.npy



E:\crew2_train\validation\training_data-95.npy




______________________________
Weights: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Epoch7 weights' accuracy: 0.547. Accuracy considering 'closeness': 0.603
{0: 11630, 1: 100, 2: 589, 3: 1426, 4: 2183, 5: 2433, 6: 30, 7: 155, 8: 1978}


In [47]:
from grabscreen import grab_screen
import time
from directkeys import PressKey,ReleaseKey, W, A, S, D
from getkeys import key_check
from collections import deque, Counter
import random
from statistics import mode,mean

In [48]:
GAME_WIDTH = 1280
GAME_HEIGHT = 720

how_far_remove = 800
rs = (20,15)
log_len = 25

motion_req = 800
motion_log = deque(maxlen=log_len)

WIDTH = 480
HEIGHT = 270
LR = 1e-3
EPOCHS = 10

choices = deque([], maxlen=5)
hl_hist = 250
choice_hist = deque([], maxlen=hl_hist)

w  = [1,0,0,0,0,0,0,0,0]
s  = [0,1,0,0,0,0,0,0,0]
a  = [0,0,1,0,0,0,0,0,0]
d  = [0,0,0,1,0,0,0,0,0]
wa = [0,0,0,0,1,0,0,0,0]
wd = [0,0,0,0,0,1,0,0,0]
sa = [0,0,0,0,0,0,1,0,0]
sd = [0,0,0,0,0,0,0,1,0]
nk = [0,0,0,0,0,0,0,0,1]

t_time = 0.25

def straight():
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(D)
    ReleaseKey(S)

def left():
    if random.randrange(0,3) == 1:
        PressKey(W)
    else:
        ReleaseKey(W)
    PressKey(A)
    ReleaseKey(S)
    ReleaseKey(D)
    #ReleaseKey(S)

def right():
    if random.randrange(0,3) == 1:
        PressKey(W)
    else:
        ReleaseKey(W)
    PressKey(D)
    ReleaseKey(A)
    ReleaseKey(S)
    
def reverse():
    PressKey(S)
    ReleaseKey(A)
    ReleaseKey(W)
    ReleaseKey(D)


def forward_left():
    PressKey(W)
    PressKey(A)
    ReleaseKey(D)
    ReleaseKey(S)
    
    
def forward_right():
    PressKey(W)
    PressKey(D)
    ReleaseKey(A)
    ReleaseKey(S)

    
def reverse_left():
    PressKey(S)
    PressKey(A)
    ReleaseKey(W)
    ReleaseKey(D)

    
def reverse_right():
    PressKey(S)
    PressKey(D)
    ReleaseKey(W)
    ReleaseKey(A)

def no_keys():

    if random.randrange(0,3) == 1:
        PressKey(W)
    else:
        ReleaseKey(W)
    ReleaseKey(A)
    ReleaseKey(S)
    ReleaseKey(D)

In [49]:
## Reload weights & model structure:

# with open('first_batch_model_Epoch_7_json.json','r') as f:
#     model_json = json.load(f)

# model = model_from_json(model_json)
# model.load_weights('first_batch_model_Epoch_7.h5')

We have loaded a previous model!!!!


In [54]:
from motion import motion_detection

In [55]:
main()       

4
3
2
1
loop took 0.526 seconds. Motion: 0. Choice: left


error: OpenCV(3.4.1) C:\projects\opencv-python\opencv\modules\core\src\arithm.cpp:683: error: (-5) When the input arrays in add/subtract/multiply/divide functions have different types, the output array type must be explicitly specified in function cv::arithm_op


In [ ]:
waa

In [ ]:
t